In [1]:
#input data 전처리
from konlpy.tag import Kkma
kkma = Kkma()
input_text = '스마트폰'
input_sim = 20  # input data 유사성 민감도 지정 / 숫자가 작을수록 관련 없는게 나올 확률이 커짐 / 최소 50이상 설정
input_text_list = input_text.split(' ') # input data 띄어쓰기로 나누기
noun_list = kkma.nouns(input_text)

# kkma로 명사 추출시 input_text 중복해서 들어가기 때문에 제거
if len(input_text_list) == len(noun_list) :
    text = input_text

else :
    [i for i in input_text_list if not i in noun_list or noun_list.remove(i)] #차집합인데 순서가 안 바뀜 
    text = ' '.join(noun_list)
print(text)

#Kkma 처음에 불러오는데 10초

스마트 폰


In [2]:
#슬로건 생성 모델 실행
import requests
import json


r = requests.post(
    'https://train-8dgtlge21881yafjrqb4-gpt2-train-teachable-ainize.endpoint.ainize.ai/predictions/gpt-2-ko-small-finetune', #슬로건 , 없는 정제된 5에포크
    headers = {'Content-Type' : 'application/json'
               },
    data=json.dumps({
  "text": text,
  "num_samples": 200,
  "length": 20
    }))

#슬로건 추가
slogan_list = []
for slogan in r.json():
    slogan = slogan.split('\n')[0]
    slogan = slogan.split(',')[1:]
    slogan = ', '.join(slogan)
    if slogan :
      slogan_list.append(slogan)
      print(slogan)
print(len(slogan_list),'개 완료')

#슬로건 데이터 200개 생성 25초

이건 아니다
세계 어디서도 쉽게,  편하게
당신의 기분좋은 첫 번째 폰
BEM
나만의 폰을 완성하다
누군가는 다 아는 폰을 경험하라
HDMI로 답할 수 없는 놀라운 세상이 시작됐다
지금까지 없었던 스마트 플랫폼
Give LTE,  Lets like your phone
그녀에게 폰을 선물하라
Live Go
한 손에 들고 두 손으로 가볍게
이제 그 폰도 당신이 할 수 있도록
가까이에서 즐길 수 있는 새로운 경험
It's keep you are to be on.
그냥,  그 폰에서
모든 게 새롭게
대한민국 1등 금융
우리의 선택은 당신 뿐입니다
요금폭탄 톡,  폰은 폰이다
나만의 S펜으로 편리하게 셀프폰을 만듭니다
세계최초 5G 스마트폰
휴대폰이 아니다,  폰이다
당신의 개성을 더 세게,  더 깊이
다들 놀라지만
스펙을 키우는 시대
Smart machine
모두의 마음 그대로
새로운 시대의 시작
폰의 시대를 만듭니다
이제 그 폰은 당신을 향해
갤럭시 폴드2와 함께라면
세상이 원하는 폰케이스
스마트폰은 언제나 즐겁다
다시,  이게 할일
요즘 딱
AR,  OPEN
한번 더 가까이. 더 멀리
요즘은 MP3도 할 수 있을 거예요?
당신의 첫 번째 스마트폰
폰의 끝
My Runset Around to Live
I hope you show
SKT만의 LTE 기술
PLAYS ME
세상의 모든 폰을 하나로
모바일 기기 그 이상을
스마트 폰 개인정보의 수집 관리 서비스, 스스로 지키는
당신이 더 편리하게 느껴집니다
지금,  우리가 살고 있는 일상은 예술이 아닙니다
Timeless Amazing. This party
그 어떤 스마트폰보다 편리하고,  완벽한
아이폰5S 옵티머스G 스마트폰, 차원이 다른 새로운 즐거
당신을 더 가깝게
요즘에는 폰으로 통한다
내 생각 그대로
이것이 SK텔레콤 5G의 미래다
휴대폰도 모바일 시대
갤럭시S3 스마트폰, 더 특별하게
요즘 아이들 눈엔
아무 것도 아닌 일상의 편안함
My Good
스마트폰을 넘나드는 새로운 혁신
당신만의 폰으로 당신의 마음을 사로

In [3]:
#차집합 함수
def differ_sets(a,b) : 
    lst = list(set(a) - set(b))
    return lst
    

In [4]:
#영어 슬로건 따로 추출
import re

eng_list = []
for slogan in slogan_list :
    slogan_ = re.sub('[^A-Za-z가-힣]', '',slogan) #영어 한글만 남기기
    slogan_ = re.sub('[^가-힣]',' ', slogan_) #한글이 있으면 공백 채우기
    if slogan_.isspace():    #isalpha()는 영어 또는 한글 유무를 찾아서 안 됨
        eng_list.append(slogan)
        
print(eng_list)

#차집합 
kor_list = differ_sets(slogan_list, eng_list) #한국 슬로건만 있는 리스트

['BEM', 'Give LTE,  Lets like your phone', 'Live Go', "It's keep you are to be on.", 'Smart machine', 'AR,  OPEN', 'My Runset Around to Live', 'I hope you show', 'PLAYS ME', 'Timeless Amazing. This party', 'My Good', 'TIME TRIKE ID', 'STAY FREE EXO', 'My hob Key to good', 'I Want to Mobile?', 'Premium Pack', 'Moment size momen', "What's do?", 'PLAY ONE UP', 'Life Toneras', 'AMORE MORROW', 'Go Together', 'SM3,  To Something']


In [5]:
#문장 유사도
#개별 추출
from sentence_transformers import SentenceTransformer, util
import numpy as np
import random

#모델 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

#유사도 비교할 리스트
corpus = kor_list
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)


#비교할 슬로건 선택 
confirm_list = [] # 슬로건 추출
n = 0
try : #n이 증가하지 않을 경우 무한루프? 
    while n < 10 :
        query = random.sample(kor_list, 1)
        # print('='*40)
        # print("Query : ", query)
        kor_list = differ_sets(kor_list, query)  #kor_list에서 query를 제거 
        
        #코사인 유사도 사용하여 5개 유사한 슬로건 찾기
        top_k = 6 #query 포함 top 5개
        query_embedding = model.encode(query, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        cos_scores = cos_scores.cpu()
        top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k] # np 사용 이유 : 순위를 순서대로 맞추기 위함
        
        #민감도 비교하기 위한 유사도 더하기      
        sum = 0
        for idx in top_results[1:top_k]:
            sum += cos_scores[idx]
        f_sum = float(sum)/5 #tensor to float
        # print(f_sum)
        
        #사용자 인풋 민감도 비교    
        if f_sum >= input_sim / 100 :
            confirm_list.append(query)
            n += 1
            #print(len(kor_list))
            
        else : 
            no_sim_list.append(query)
            print('관련이 없는 슬로건 데이터 추가')
  
                
                
except :
    print('데이터가 부족합니다.')
    
print('완료')
print(confirm_list)

#슬로건 25개 보여주는데 40초 / 모델 불러들어오는 시간 포함 => 다시 시작되면 2~5초

데이터가 부족합니다.
완료
[['SK텔레콤의 첫번째 ARM ALL NEW 스마트폰이 될'], ['휴대폰이 아니다,  그건 폰이 아닙니다'], ['그녀에게 폰을 선물하라'], ['폰의 끝'], ['당신의 첫 번째 스마트폰'], [' 당신의 폰은 선택의 차이가 큰 것이']]
